<a href="https://colab.research.google.com/github/NikolasGialitsis/LDA2vec/blob/master/LDA2vec_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [304]:
!unzip /content/lda2vec.zip

unzip:  cannot find or open /content/lda2vec.zip, /content/lda2vec.zip.zip or /content/lda2vec.zip.ZIP.


In [305]:
!pip install spacy
!pip install jellyfish
!pip install -r /content/lda2vec/requirements.txt 
!pip install pylda2vec

  Using cached https://files.pythonhosted.org/packages/3d/fc/110649d12075cd6627113ea5fb6cab1a056a2676113c31c6b3bc7f4086ef/cupy-7.3.0.tar.gz
ERROR: Could not find a version that satisfies the requirement en_core_web_md (from -r /content/lda2vec/requirements.txt (line 6)) (from versions: none)
ERROR: No matching distribution found for en_core_web_md (from -r /content/lda2vec/requirements.txt (line 6))


In [306]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [307]:
%cd /content/lda2vec/
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec
from lda2vec import preprocess, Corpus
#changed Preprocess.py line 'nlp = spacy.load("en_core_web_sm")' to solve error en not found
#and removed the import ... as en line 

import logging
logging.basicConfig()
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus

/content/lda2vec


In [308]:
gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU:" + str(gpu_id))

Using GPU:0


In [309]:
import nltk 
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [310]:

import json
with open('/content/new_document.json') as f:
  data = json.loads(f.read())
titles = []
abstracts = []
concats = []
for entry in data:
  concat = (entry['joint_text']).split('------------------------------')  
  title = concat[0]
  abstract = concat[1]
  print('title:',title)
  print('abstract',abstract[:100])
  concats.append(concat[0] + ' ' + concat[1])
  titles.append(title)
  abstracts.append(abstract)

stop_words = set(stopwords.words('english')) 
new_text = []
for text in concats.copy():
  word_tokens = word_tokenize(text) 
  #remove stopwords
  text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
  #remove punctuation
  #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

  text = ''.join([w for w in text if w not in string.punctuation])
  new_text.append(text) 

max_length = 10000
tokens , vocab = preprocess.tokenize(new_text, max_length, merge=False,n_threads=4)





title: [The treatment proposal for the patients with breast diseases in the central epidemic area of 2019 coronavirus disease].

abstract 
Currently, the epidemic of 2019 coronavirus disease (COVID-19) is still ongoing. The characteristic


In [0]:
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=0)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)

In [312]:
%cd /content
fn_weights = 'weights.pkl'
fn_sampler = 'sampler.pkl'
fn_factors = 'factors.pkl'
weights = pickle.load(open(fn_weights, 'rb'))
sampler = pickle.load(open(fn_sampler, 'rb'))
factors = pickle.load(open(fn_factors, 'rb'))

vocab = pickle.load(open('vocab.pkl', 'rb'))

/content


In [0]:
fn_vocab = 'vocab.pkl'
fn_corpus = 'corpus.pkl'
fn_flatnd = 'flattened.npy'
fn_docids = 'doc_ids.npy'

In [340]:
# Model Parameters
# Number of documents
n_docs = doc_ids.max() + 1
# Number of unique words in the vocabulary
n_vocab = flattened.max() + 1
# 'Strength' of the dircihlet prior; 200.0 seems to work well
clambda = 200.0
# Number of topics to fit
n_topics = int(os.getenv('n_topics', 10))
batchsize = 16
# Power for neg sampling
power = float(os.getenv('power', 0.75))
# Intialize with pretrained word vectors
pretrained = bool(int(os.getenv('pretrained', True)))
# Sampling temperature
temperature = float(os.getenv('temperature', 1.0))
# Number of dimensions in a single word vector
n_units = int(os.getenv('n_units', 30))
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_vocab]
print(words)
# How many tokens are in each document
doc_idx, lengths = np.unique(doc_ids, return_counts=True)
doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
doc_lengths[doc_idx] = lengths

# Count all token frequencies
tok_idx, freq = np.unique(flattened, return_counts=True)
term_frequency = np.zeros(n_vocab, dtype='int32')
term_frequency[tok_idx] = freq

['out_of_vocabulary', 'out_of_vocabulary', '<SKIP>', 'covid19', 'patients', 'coronavirus', 'disease', 'cases', '2019', 'china', 'sarscov2', 'clinical', 'health', 'respiratory', 'infection', 'outbreak', 'severe', 'novel', '2020', 'wuhan', 'epidemic', 'treatment', 'pneumonia', 'virus', 'acute', 'results', 'transmission', 'syndrome', 'control', 'confirmed', 'data', 'february', 'spread', 'risk', 'days', 'study', 'public', 'case', 'symptoms', 'methods', 'reported', 'care', 'infected', 'viral', 'number', 'management', 'may', 'pandemic', 'first', 'measures', 'group', 'countries', 'also', 'prevention', 'early', 'january', 'characteristics', 'caused', 'diagnosis', 'patient', 'findings', 'medical', 'emergency', 'new', '2019ncov', 'world', 'hubei', 'among', 'including', 'time', 'enditag', 'itag', 'hospital', 'december', 'chest', 'two']


In [0]:

model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                n_samples=15, power=power, temperature=temperature)
vectors = np.load('vectors.npy')
model.sampler.W.data[:, :] =  vectors[:n_vocab, :]

In [0]:
fn_vocab = 'vocab.pkl'
fn_corpus = 'corpus.pkl'

vocab = pickle.load(open(fn_vocab, 'rb'))
corpus = pickle.load(open(fn_corpus, 'rb'))
old_flattened = np.load(fn_flatnd)
n_vocab = old_flattened.max() + 1
words = corpus.word_list(vocab)[:n_vocab]



In [343]:


#model.sampler.W.data[:, :] = new_text[:n_vocab, :]

data = prepare_topics(cuda.to_cpu(weights.copy()),
                      cuda.to_cpu(factors.copy()),
                      cuda.to_cpu(sampler.copy()),
                      words)
#add new randomized vector for new doc

random_doc_topic = np.random.dirichlet(np.ones(10),size=1)
first_doc = data['doc_topic_dists'][0]

num_docs = data['doc_topic_dists'].shape[0]
data['doc_topic_dists'] = np.append(arr=data['doc_topic_dists'],values=random_doc_topic)
data['doc_topic_dists'] = data['doc_topic_dists'].reshape(num_docs+1,n_topics)


print(data['doc_topic_dists'].shape)
print('random_doc = ',random_doc_topic)


(745, 10)
random_doc =  [[0.0663702  0.15741455 0.07266566 0.14981045 0.00932821 0.26088289
  0.01057492 0.13455583 0.05626844 0.08212885]]


In [0]:
model.to_gpu()
optimizer = O.Adam()
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)
j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')


In [345]:
n_epochs = 1500

for epoch in range(1):
    data = prepare_topics(cuda.to_cpu(weights.copy()),
                          cuda.to_cpu(factors.copy()),
                          cuda.to_cpu(sampler.copy()),
                          words)
    #top_words = print_top_words_per_topic(data)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    np.savez('topics.pyldavis', **data)
    #print(data)
    for d, f in utils.chunks(batchsize, np.array(doc_ids), flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy(),update_only_docs=True)
        print(l)
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1

3356.1287
after partial fitting: 3356.1287
J:00000 E:00000 L:3.356e+03 P:-2.090e+01 R:2.462e+02
20219.592
after partial fitting: 20219.592
J:00001 E:00000 L:2.022e+04 P:-2.090e+01 R:2.756e+02
17371.99
after partial fitting: 17371.99
J:00002 E:00000 L:1.737e+04 P:-2.090e+01 R:3.497e+02
17151.164
after partial fitting: 17151.164
J:00003 E:00000 L:1.715e+04 P:-2.090e+01 R:3.360e+02
11744.21
after partial fitting: 11744.21
J:00004 E:00000 L:1.174e+04 P:-2.090e+01 R:3.164e+02
21842.562
after partial fitting: 21842.562
J:00005 E:00000 L:2.184e+04 P:-2.090e+01 R:3.395e+02
20980.445
after partial fitting: 20980.445
J:00006 E:00000 L:2.098e+04 P:-2.090e+01 R:3.378e+02


In [326]:
print('==== MAKE SURE THAT THE OTHER DOCS DONT CHANGE')
print(first_doc)
print(data['doc_topic_dists'][0])
print('\n==================================')
print('==== MAKE SURE THAT THE NEW DOC HAS CHANGED')
print(random_doc_topic)
print(data['doc_topic_dists'][-1])

==== MAKE SURE THAT THE OTHER DOCS DONT CHANGE
[0.10384709 0.08276632 0.10338711 0.07900679 0.1079841  0.11230687
 0.09587169 0.10159854 0.1013382  0.11189319]
[0.10384709 0.08276632 0.10338711 0.07900679 0.1079841  0.11230687
 0.09587169 0.10159854 0.1013382  0.11189319]

==== MAKE SURE THAT THE NEW DOC HAS CHANGED
[[0.08702473 0.09879306 0.07320704 0.32777722 0.14274596 0.16095911
  0.0242389  0.02595778 0.02104086 0.03825533]]
[0.09601615 0.09627663 0.10512727 0.09975246 0.11829065 0.09382185
 0.09439588 0.10970131 0.08970814 0.09690975]


In [327]:
t=0
print('New Document Composition\n')
for d in data['doc_topic_dists'][-1]:
  print('\t',round(d*100,2),'\t percent of Topic',t)
  t+=1 

New Document Composition

	 9.6 	 percent of Topic 0
	 9.63 	 percent of Topic 1
	 10.51 	 percent of Topic 2
	 9.98 	 percent of Topic 3
	 11.83 	 percent of Topic 4
	 9.38 	 percent of Topic 5
	 9.44 	 percent of Topic 6
	 10.97 	 percent of Topic 7
	 8.97 	 percent of Topic 8
	 9.69 	 percent of Topic 9


In [0]:
pyLDAvis.display(prepared_data)